<a href="https://colab.research.google.com/github/brendanlooker/colab-examples/blob/main/dataform/dataformAPI_workflowInvocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Client Library

In [ ]:
!pip install google-cloud-dataform

In [ ]:
from google.cloud import dataform_v1beta1

In [ ]:
# Authenticate via ADC (Application Default Credentials)

!gcloud auth application-default login


In [ ]:
# Complete set-up
project_id = "" # @param {type:"string"}

!gcloud auth application-default set-quota-project {project_id}

In [ ]:


# Config
region = "europe-west4"      # @param {type:"string"}
repository_id = "dataform-demo" # @param {type:"string"}

def list_recent_workflow_invocations(project_id, region, repository_id, limit=5):
    """Lists recent Dataform workflow invocation IDs."""
    client = dataform_v1beta1.DataformClient()

    parent = f"projects/{project_id}/locations/{region}/repositories/{repository_id}"

    try:
        request = dataform_v1beta1.ListWorkflowInvocationsRequest(
            parent=parent,
            page_size=limit,
        )
        response = client.list_workflow_invocations(request=request)

        invocations = []
        for invocation in response:
            invocation_id = invocation.name.split("/")[-1]
            invocations.append({
                "id": invocation_id,
                "state": invocation.state.name,
                "invocation_path": invocation.name
            })

        return invocations

    except Exception as e:
        print(f"Error listing workflow invocations: {e}")
        return []


invocations = list_recent_workflow_invocations(project_id, region, repository_id)

for i, invocation in enumerate(invocations):
    print(f"\n🔹 Invocation {i + 1}")
    print(f"ID: {invocation['id']}")
    print(f"Status: {invocation['state']}")
    print(f"Full Path: {invocation['invocation_path']}")


In [ ]:


# Config Workflow Invocation Id

workflow_invocation_id = "1742493924-a70d38d7-4df2-48b6-8798-bc1bc997721c" #@param {type:"string"} # Replace as required

def get_dataform_workflow_invocation_details(project_id, region, repository_id, workflow_invocation_id):
    """Retrieves Dataform workflow invocation details, including failed SQLX actions.

    Returns:
        dict: Workflow details, including failed SQLX job names and error messages.
    """

    client = dataform_v1beta1.DataformClient()

    # Define workflow invocation path
    workflow_invocation_name = f"projects/{project_id}/locations/{region}/repositories/{repository_id}/workflowInvocations/{workflow_invocation_id}"

    try:
        # Step 1: Get Workflow Invocation Status
        workflow_invocation = client.get_workflow_invocation(name=workflow_invocation_name)

        details = {
            "status": workflow_invocation.state.name,  # Overall workflow status
            "compilation_result": workflow_invocation.compilation_result,
            "failed_actions": [],
        }

        # Step 2: Query Workflow Invocation Actions
        action_request = dataform_v1beta1.QueryWorkflowInvocationActionsRequest(
            name=workflow_invocation_name
        )
        action_results = client.query_workflow_invocation_actions(request=action_request)

        # Step 3: Extract Failed Actions
        for action in action_results.workflow_invocation_actions:
            if action.state.name == "FAILED":
                details["failed_actions"].append({
                    "sqlx_reference": action.target.name if action.target else "Unknown",
                    "failure_reason": action.failure_reason if action.failure_reason else "No error message provided",
                })

        return details

    except Exception as e:
        print(f"Error getting workflow invocation details: {e}")
        return None


details = get_dataform_workflow_invocation_details(project_id, region, repository_id, workflow_invocation_id)

if details:
    print(f"Workflow invocation status: {details['status']}")
    print(f"Compilation result: {details['compilation_result']}")

    if details["failed_actions"]:
        print("\nFailed Actions:")
        for failed in details["failed_actions"]:
            print(f"SQLX: {failed['sqlx_reference']}")
            print(f"Failure Reason: {failed['failure_reason']}")
    else:
        print("\nNo failed actions.")
else:
    print("Could not retrieve workflow invocation details.")


Workflow invocation status: FAILED
Compilation result: projects/828575085389/locations/europe-west4/repositories/dataform-demo/compilationResults/ee227ec9-1a8d-483f-8276-d65039674636

Failed Actions:
SQLX: product_fact
Failure Reason: Syntax error: Expected ")" but got identifier "joins" at [39:1]
